In [12]:
# MLP 
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

In [2]:
import mailbox

# Path to your mbox file
mbox_path = 'promotions.mbox'

# Open the mbox file
mbox = mailbox.mbox(mbox_path)

subject_data = []

# Iterate through messages in the mbox file
for message in mbox:
    # Access subject, sender, and other headers
    subject = message['subject']
    sender = message['from']
    date = message['date']
    
    subject_data.append(subject)

# Close the mbox file
mbox.close()

In [23]:
def clean_encoded_subjects(subjects):
    cleaned_subjects = []

    for subject in subjects:
        try:
            # Decode the subject
            decoded_fragments = decode_header(subject)
            decoded_subject = ''

            for fragment, encoding in decoded_fragments:
                if fragment is None:
                    continue  # Skip any None fragments
                if isinstance(fragment, bytes):
                    # Decode fragment with the provided encoding if given, otherwise assume utf-8
                    encoding = encoding or 'utf-8'
                    fragment = fragment.decode(encoding)
                decoded_subject += fragment

            # Replace MIME-specific underscore characters with spaces
            decoded_subject = decoded_subject.replace('_', ' ')
            cleaned_subjects.append(decoded_subject)
        except Exception as e:
            print(f"Skipping subject due to error: {e}")
            # Optionally, append a placeholder or log the error
            cleaned_subjects.append("Subject decoding failed")

    return cleaned_subjects

cleaned_subjects = clean_encoded_subjects(subject_data)

Skipping subject due to error: expected string or bytes-like object, got 'NoneType'
Skipping subject due to error: unknown encoding: unknown-8bit
Skipping subject due to error: unknown encoding: unknown-8bit
Skipping subject due to error: unknown encoding: unknown-8bit
Skipping subject due to error: unknown encoding: unknown-8bit
Skipping subject due to error: unknown encoding: unknown-8bit
Skipping subject due to error: unknown encoding: unknown-8bit
Skipping subject due to error: unknown encoding: unknown-8bit


In [25]:
import numpy as np

length_data = len(cleaned_subjects)

email_open_rates = np.random.rand(length_data)

In [26]:
len(cleaned_subjects)

30211

In [27]:
len(email_open_rates)

30211

In [31]:
# Tokenize and vectorize the text data
tokenizer = Tokenizer(num_words=5000) # 5000 most common words considered - will test adjusting this later - probably more words needed
tokenizer.fit_on_texts(cleaned_subjects)
sequences = tokenizer.texts_to_sequences(cleaned_subjects)
data = pad_sequences(sequences, maxlen=100) # Truncates to first 100 characters - should be fine for email subject lines

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, email_open_rates, test_size=0.2, random_state=42)

In [32]:
# Define the MLP architecture
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(100,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [33]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')  # Regression loss function

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
756/756 [==============================] - 4s 4ms/step - loss: 0.3333 - val_loss: 0.3311
Epoch 2/10
756/756 [==============================] - 2s 3ms/step - loss: 0.3323 - val_loss: 0.3318
Epoch 3/10
756/756 [==============================] - 3s 3ms/step - loss: 0.3327 - val_loss: 0.3353
Epoch 4/10
756/756 [==============================] - 2s 3ms/step - loss: 0.3320 - val_loss: 0.3321
Epoch 5/10
756/756 [==============================] - 2s 3ms/step - loss: 0.3323 - val_loss: 0.3334
Epoch 6/10
756/756 [==============================] - 3s 3ms/step - loss: 0.3361 - val_loss: 0.3314
Epoch 7/10
756/756 [==============================] - 3s 3ms/step - loss: 0.3345 - val_loss: 0.3310
Epoch 8/10
756/756 [==============================] - 3s 3ms/step - loss: 0.3356 - val_loss: 0.3312
Epoch 9/10
756/756 [==============================] - 3s 4ms/step - loss: 0.3325 - val_loss: 0.3310
Epoch 10/10
756/756 [==============================] - 2s 3ms/step - loss: 0.3330 - val_loss: 0.3333

In [35]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")

# Predict continuous values on new data
predictions = model.predict(X_test)

189/189 [==============================] - 0s 2ms/step - loss: 0.3333
Test Loss: 0.3333
189/189 [==============================] - 0s 2ms/step
